# Dinamic Dashboards

In [36]:
import dash
import pandas as pd
import plotly.express as px
from dash import dcc, Dash, html
from dash.dependencies import Input, Output

In [71]:
df = pd.read_csv('intro_bees.csv')
df.head(2)

,Program,Year,Period,State,ANSI,Affected by,Pct of Colonies Impacted,state_code
0,SURVEY,2019,JAN THRU MAR,Alabama,1,Disease,1.8,AL
1,SURVEY,2019,JAN THRU MAR,Alabama,1,Other,3.1,AL


In [73]:

# Step 1: Initialize the Dash app
app = Dash(__name__)

# Step 2: Define App Layout
app.layout = html.Div([
    # Title
    html.H1('Just a Dashboard', style={'textAlign': 'center', 'color': 'blue'}),

    # Paragraph for Description
    html.P('Paragraph has been made',
           style={'fontSize': '18px', 'textAlign': 'center'}),

    # Dropdown for Category Selection
    html.Div([
        html.Label('Select a category', style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='category_dropdown',
            options=[{'label': cat, 'value': cat} for cat in df['State'].unique()],
            value='A',  # Default value
            clearable=True,
            style={'width': '50%'}
        )
    ], style={'marginBottom': '20px'}),

    # Slider for Numeric Filtering
    html.Div([
        html.Label('Select a Number:', style={'fontWeight': 'bold'}),
        dcc.Slider(
            id='number_slider',
            min=df['ANSI'].min(),
            max=df['ANSI'].max(),
            step=1,
            value=df['ANSI'].max(),  # Default Slider position
            marks={i: str(i) for i in range(df['ANSI'].min(), df['ANSI'].max() + 1)}
        )
    ], style={'marginBottom': '20px'}),

    # Date Picker
    html.Div([
        html.Label('Select Date: ', style={'fontWeight': 'bold'}),
        dcc.DatePickerRange(
            id='date_picker',
            start_date=df['Year'].min(),
            end_date=df['Year'].max(),
            display_format='YYYY-MM-DD'
        )
    ], style={'marginBottom': '20px'}),

    # Input Box
    html.Div([
        html.Label('Enter a Text Label: ', style={'fontWeight': 'bold'}),
        dcc.Input(
            id='text_input',
            type='text',
            value='Default Label',
            style={'marginRight': '10px'}
        )
    ], style={'marginBottom': '20px'}),

    # Graph to Display
    dcc.Graph(id='bar_chart'),

    # Interval for Auto Update
    dcc.Interval(
        id='interval_update',
        interval=5 * 1000,
        n_intervals=0
    )
])

# Step 3: Define Callback for Interactivity
@app.callback(
    Output('bar_chart', 'figure'),
    [
        Input('category_dropdown', 'value'),
        Input('number_slider', 'value'),
        Input('date_picker', 'start_date'),
        Input('date_picker', 'end_date'),
        Input('text_input', 'value')
    ]
)
def update_chart(selected_category, max_value, start_date, end_date, text_label):
    # Filter data based on user input
    filtered_df = df[
        (df['State'] == selected_category) &
        (df['ANSI'] <= max_value) &
        (df['Year'] >= pd.to_datetime(start_date)) &
        (df['Year'] <= pd.to_datetime(end_date))
    ]

    # Create a bar chart using filtered data
    figure = {
        'data': [
            {
                'x': filtered_df['Year'],
                'y': filtered_df['ANSI'],
                'type': 'bar',
                'name': text_label
            }
        ],
        'layout': {
            'title': f'Data for {text_label}',
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'ANSI Value'}
        }
    }
    return figure

# Step 4: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)
